In [2]:
"""
Play around with getting Mallet LDA to run
"""

from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.wrappers import LdaMallet
from gensim import corpora

path_to_mallet_binary = "/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/mallet-2.0.8/bin/mallet"
#model = LdaMallet(path_to_mallet_binary, corpus=common_corpus, num_topics=20, id2word=common_dictionary)
#vector = model[common_corpus[0]]  # LDA topics of a documents

In [ ]:
"""
from gensim import corpora
reader = open("../115_legislation_lines.txt")
lines = [line.replace("\n", "") for line in reader.readlines()]
print(len(lines))
texts = [line.split(" ") for line in lines]
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/115_legislation.dict')  # store the dictionary, for future reference
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]
"""

In [ ]:
#model = LdaMallet(path_to_mallet_binary, corpus=corpus, num_topics=20, id2word=dictionary)

In [ ]:
#model.print_topics(30)

In [ ]:
#model = LdaMallet(path_to_mallet_binary, corpus=corpus, num_topics=50, id2word=dictionary)

In [ ]:
#response = model.show_topics(50, formatted=False)
#print(response)

In [ ]:
#covid_model = LdaMallet.load("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/models/covid-50.model")

In [9]:
import random

def setup_word_intrusion(topics_list, n=20, topic_idxs=None):
    """
    topics_list: format is a list of tuples [
    (topic_id, [(word, score),(word, score),(word, score)]),
    (topic_id, [(word, score),(word, score),(word, score)])]
    This is the format given out of the gensim wrapper for Mallet
    
    n: The number of topics to sample
    """
    # Can't sample more than the available topics
    assert len(topics_list) >= n
    
    intruder_list = []
    # Generate n random ints for the selection of topics we'll conduct intrusion on
    if not topic_idxs:
        topic_idxs = random.sample(range(len(topics_list)), n)
        
    for topic_idx in topic_idxs:
        # select another topic from which to grab a term, exclude the current topic
        random_topic_idx = random.choice([idx for idx in range(0, len(topics_list)) if (idx != topic_idx and idx not in topic_idxs)])

        # take the top 5 words of the current topic and ONE of the top 5 terms from the top of the other topic
        # assert that the new word is not in the top 50 words of the original topic
        correct_words = [word_tuple[0] for word_tuple in topics_list[topic_idx][1][:5]]
        
        # This collects the top 50 words of the current topic
        top_topic_words = [word_tuple[0] for word_tuple in topics_list[topic_idx][1][:50]]

        # This collects the top words of the 'intruder' topics that do NOT overlap with any of the top
        # 10 words of the other topic
        top_random_words = [word_tuple[0] for word_tuple in topics_list[random_topic_idx][1][:5] \
                            if word_tuple[0] not in top_topic_words]
        
        # EDGE-CASE - The top 50 words of the selected topic may overlap heavily with the
        # 'intruder' topics's top words. In this case, narrow down the set of excluded terms
        # for the current topic to just the top 10. If that doesn't work, then..... skip??
        if not top_random_words:
            top_topic_words = [word_tuple[0] for word_tuple in topics_list[topic_idx][1][:10]]
            top_random_words = [word_tuple[0] for word_tuple in topics_list[random_topic_idx][1][:5] \
                                if word_tuple[0] not in top_topic_words]
        
            if not top_random_words:
                print(f"Skipping word intrusion for topic {topic_idx} with intruder {random_topic_idx}")
                continue
        # select the intruder word
        selected_intruder = random.choice(top_random_words)
        
        print(topic_idx, random_topic_idx, correct_words + [selected_intruder])
        
        # The last word in each list is the 'intruder', this should be randomized before showing
        intruder_list.append([topics_list[topic_idx][0]] + correct_words + [selected_intruder])
    return intruder_list
    
#covid_bad_topics = [33, 40]
#covid_topics = covid_model.show_topics(num_topics=50, num_words=50, formatted=False)
#covid_topics = [covid_topics[idx] for idx in range(0, len(covid_topics)) if idx not in covid_bad_topics]

#covid_intruders = setup_word_intrusion(covid_topics)

In [7]:
import csv
def load_topics(path):
    
    reader = csv.DictReader(open(path), delimiter=",")
    topics_list = [(topic_name, []) for topic_name in reader.fieldnames if topic_name != "Word"]
    for idx, row in enumerate(reader):
        for topic_tuple in topics_list:
            topic_name = topic_tuple[0]
            topic_tuple[1].append((row["Word"], row[topic_name]))

    # We need to sort each of the word lists once they are reassembled
    for topic_tuple in topics_list:
        topic_tuple[1].sort(key=lambda t: t[1])
    return topics_list


topics_list = load_topics("/Users/hiancheong/Downloads/export_shro_10topics/word_topics.csv")
topic_intruders = setup_word_intrusion(topics_list, n=5)

3 7 ['_survivor', 'ability', 'active', 'activity', 'aerobic', 'able']
8 7 ['asthma', 'day', 'diet', 'eating', 'healthy', 'ability']
4 8 ['aerobic', 'allergies', 'body', 'concerns', 'condition', 'eating']
5 2 ['arthritis', 'aspects', 'fairly', 'overweight', 'related', 'taking']
1 3 ['biking', 'chronic', 'conditions', 'fairly', 'family', 'ability']


In [ ]:
legislation_model = LdaMallet.load("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/models/legislation-50.model")

In [ ]:
#legislation_topics = legislation_model.show_topics(num_topics=50, num_words=50, formatted=False)
#legislation_intruders = setup_word_intrusion(legislation_topics)

In [12]:
big_covid_model = LdaMallet.load("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/models/covid-150.model")
big_leg_model = LdaMallet.load("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/models/legislation-100.model")
news_model = LdaMallet.load("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/models/20_news_20.model")

In [13]:
covid_bad_big_topics = [4,9,12]
big_covid_topics = big_covid_model.show_topics(num_topics=150, num_words=50, formatted=False)
big_covid_topics = [big_covid_topics[idx] for idx in range(0, len(big_covid_topics)) if idx not in covid_bad_big_topics]

big_leg_topics = big_leg_model.show_topics(num_topics=100, num_words=50, formatted=False)

news_topics = news_model.show_topics(num_topics=20, num_words=20, formatted=False)

In [14]:
"""
Identify the set of indices to draw topics from
"""
N = 30
covid_topic_idxs = random.sample(range(len(big_covid_topics)), N)
leg_topic_idxs = random.sample(range(len(big_leg_topics)), N)
news_topic_idxs = random.sample(range(len(news_topics)), 10)

In [16]:
big_covid_intruders = setup_word_intrusion(big_covid_topics, topic_idxs=covid_topic_idxs)
big_leg_intruders = setup_word_intrusion(big_leg_topics, topic_idxs=leg_topic_idxs)
news_intruders = setup_word_intrusion(news_topics, topic_idxs=news_topic_idxs)

24 29 ['compounds', 'reaction', 'compound', 'activity', 'derivatives', 'golgi']
12 138 ['inflammatory', 'cytokines', 'response', 'inflammation', 'immune', 'surface']
64 36 ['testing', 'test', 'tests', 'sensitivity', 'diagnostic', 'information']
38 122 ['recipients', 'transplant', 'transplantation', 'donor', 'rejection', 'learning']
134 27 ['food', 'products', 'risk', 'production', 'safety', 'world']
54 15 ['trauma', 'injury', 'bone', 'injuries', 'pain', 'species']
103 94 ['cancer', 'tumor', 'tumors', 'breast', 'cell', 'recombinant']
86 124 ['water', 'treatment', 'surface', 'inactivation', 'uv', 'dna']
136 50 ['symptoms', 'fever', 'acute', 'respiratory', 'illness', 'domains']
96 135 ['data', 'system', 'information', 'user', 'users', 'clinical']
39 118 ['pedv', 'tgev', 'piglets', 'pigs', 'intestinal', 'infectious']
132 14 ['antibodies', 'epitopes', 'binding', 'peptides', 'peptide', 'class']
127 41 ['preprint', 'license', 'medrxiv', 'display', 'copyright', 'response']
143 59 ['network', '

In [17]:
def select_observed_coherence_topics(topics_list, num_topics=20, num_terms=5, topic_idxs=None):
    """
    Newman et Al - Generate a set of terms to be evaluated on a 1-3 scale ranking
    """
    if not topic_idxs:
        topic_idxs = random.sample(range(len(topics_list)), num_topics)
    topic_words = []
    for topic_idx in topic_idxs:
        # put the topic id as the first value
        topic_words.append([topics_list[topic_idx][0]] + [word_tuple[0] for word_tuple in topics_list[topic_idx][1][:num_terms]])
        
    return topic_words
select_observed_coherence_topics(news_topics, num_topics=10)

[[8, 'organization', 'subject', 'nntp', 'host', 'lines'],
 [10, 'year', 'good', 'game', 'baseball', 'team'],
 [18, 'israel', 'jews', 'israeli', 'jewish', 'people'],
 [15, 'game', 'team', 'hockey', 'play', 'games'],
 [19, 'people', 'government', 'fbi', 'gun', 'fire'],
 [9, 'lines', 'subject', 'organization', 'writes', 'article'],
 [4, 'armenian', 'turkish', 'armenians', 'world', 'people'],
 [17, 'lines', 'organization', 'subject', 'price', 'money'],
 [16, 'image', 'data', 'graphics', 'file', 'software'],
 [1, 'drive', 'card', 'mac', 'problem', 'apple']]

In [18]:
import csv
def output_to_csv(topic_terms_list, filename):
    """
    Output topics to the most friendly format for easily copying to Qualtrics
    Each column will be a new topic (either intrusion or obersered coherence)
    Requires the length of every item in topic_terms_list to be the same
    """
    columns = []
    term_len = len(topic_terms_list[0])

    for idx in range(term_len):
        row = []
        for topic_idx in range(len(topic_terms_list)):
            row.append(topic_terms_list[topic_idx][idx])
        columns.append(row)

    writer = csv.writer(open(filename, "w+"))
    for line in columns:
        writer.writerow(line)
        
    

In [ ]:
output_to_csv(news_intruders, "../news_intruders2.csv")
output_to_csv(big_leg_intruders, "../leg_intruders2.csv")
output_to_csv(big_covid_intruders, "../covid_intruders2.csv")

In [ ]:

#news_coherence = select_observed_coherence_topics(news_topics, num_topics=10, num_terms=10, topic_idxs=news_topic_idxs)
#big_leg_coherence = select_observed_coherence_topics(big_leg_topics, num_topics=30, num_terms=10, topic_idxs=leg_topic_idxs)
#big_covid_coherence = select_observed_coherence_topics(big_covid_topics, num_topics=30, num_terms=10, topic_idxs=covid_topic_idxs)

output_to_csv(news_coherence, "../news_coherence2.csv")
output_to_csv(big_leg_coherence, "../leg_coherence2.csv")
output_to_csv(big_covid_coherence, "../covid_coherence2.csv")

In [22]:
news_coherence
for topic in news_coherence:
    print(",".join(topic[1:]))
#big_leg_coherence
#big_covid_coherence
print("==LEG==")
for topic in big_leg_coherence:
    print(",".join(topic[1:]))
print("==COVID==")
for topic in big_covid_coherence:
    print(",".join(topic[1:]))

writes,article,organization,subject,lines,posting,host,nntp,distribution,usa
good,problem,subject,time,lines,organization,body,problems,long,effect
windows,file,dos,run,system,program,version,files,unix,software
key,chip,encryption,clipper,government,public,keys,security,system,law
people,government,fbi,gun,fire,president,state,police,koresh,clinton
number,health,care,national,research,medical,study,school,people,april
image,data,graphics,file,software,version,color,jpeg,format,images
god,people,jesus,christian,bible,religion,church,word,point,faith
car,cars,good,engine,article,writes,oil,bike,road,dealer
information,mail,list,send,internet,group,address,email,read,post
==LEG==
corporation,foreign,section,income,subsection,purposes,paragraph,business,year,group
environmental,federal,permit,act,application,applicable,secretary,section,review,proposed
act,bill,senate,read,introduced,session,referred,congress,cited,committee
program,secretary,programs,pilot,participating,assistance,sectio

In [ ]:
len(news_coherence)

In [ ]:

survey = {"SurveyEntry":{"SurveyID":"SV_af8BrDzrV6hF64d","SurveyName":"Topics Evaluation 1","SurveyDescription":None,"SurveyOwnerID":"UR_1MmbX0D3bAle6sR","SurveyBrandID":"umd","DivisionID":None,"SurveyLanguage":"EN","SurveyActiveResponseSet":"RS_cXYb9Itss4q723X","SurveyStatus":"Inactive","SurveyStartDate":"0000-00-00 00:00:00","SurveyExpirationDate":"0000-00-00 00:00:00","SurveyCreationDate":"2020-06-21 13:11:12","CreatorID":"UR_1MmbX0D3bAle6sR","LastModified":"2020-06-21 14:00:15","LastAccessed":"0000-00-00 00:00:00","LastActivated":"0000-00-00 00:00:00","Deleted":None},"SurveyElements":[{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"BL","PrimaryAttribute":"Survey Blocks","SecondaryAttribute":None,"TertiaryAttribute":None,"Payload":[{"Type":"Default","Description":"News Group Coherence","ID":"BL_cNPSLmmFv4ATBkN","BlockElements":[{"Type":"Question","QuestionID":"QID1"}]},{"Type":"Trash","Description":"Trash \/ Unused Questions","ID":"BL_e9vRuZf29bb60ex"},{"Type":"Standard","SubType":"","Description":"Covid Coherence","ID":"BL_7WdpPpaRap5Ti0B"},{"Type":"Standard","SubType":"","Description":"Legislation Coherence","ID":"BL_0CWrdSup3c2YS4l"},{"Type":"Standard","SubType":"","Description":"News Intruder","ID":"BL_aV5RrxWp6u04bzv","BlockElements":[{"Type":"Question","QuestionID":"QID2"}]},{"Type":"Standard","SubType":"","Description":"Covid Intruder","ID":"BL_54GKtT1Af2xXSGF"},{"Type":"Standard","SubType":"","Description":"Legislation Intruder","ID":"BL_3sfBGhcWI2RqY73"},{"Type":"Standard","SubType":"","Description":"Introduction","ID":"BL_3JEo10v03BpLhuB"}]},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"FL","PrimaryAttribute":"Survey Flow","SecondaryAttribute":None,"TertiaryAttribute":None,"Payload":{"Flow":[{"ID":"BL_3JEo10v03BpLhuB","Type":"Standard","FlowID":"FL_8"},{"ID":"BL_cNPSLmmFv4ATBkN","Type":"Block","FlowID":"FL_2"},{"ID":"BL_7WdpPpaRap5Ti0B","Type":"Standard","FlowID":"FL_3"},{"ID":"BL_0CWrdSup3c2YS4l","Type":"Standard","FlowID":"FL_4"},{"ID":"BL_aV5RrxWp6u04bzv","Type":"Standard","FlowID":"FL_5"},{"ID":"BL_54GKtT1Af2xXSGF","Type":"Standard","FlowID":"FL_6"},{"ID":"BL_3sfBGhcWI2RqY73","Type":"Standard","FlowID":"FL_7"}],"Properties":{"Count":8},"FlowID":"FL_1","Type":"Root"}},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"SO","PrimaryAttribute":"Survey Options","SecondaryAttribute":None,"TertiaryAttribute":None,"Payload":{"BackButton":"False","SaveAndContinue":"True","SurveyProtection":"PublicSurvey","BallotBoxStuffingPrevention":"False","NoIndex":"Yes","SecureResponseFiles":"True","SurveyExpiration":"None","SurveyTermination":"DefaultMessage","Header":"","Footer":"","ProgressBarDisplay":"None","PartialData":"+1 week","ValidationMessage":"","PreviousButton":" \u2190 ","NextButton":" \u2192 ","SurveyTitle":"Online Survey Software | Qualtrics Survey Solutions","SkinLibrary":"umd","SkinType":"MQ","Skin":"umd2_new","NewScoring":1}},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"SCO","PrimaryAttribute":"Scoring","SecondaryAttribute":None,"TertiaryAttribute":None,"Payload":{"ScoringCategories":[],"ScoringCategoryGroups":[],"ScoringSummaryCategory":None,"ScoringSummaryAfterQuestions":0,"ScoringSummaryAfterSurvey":0,"DefaultScoringCategory":None,"AutoScoringCategory":None}},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"PROJ","PrimaryAttribute":"CORE","SecondaryAttribute":None,"TertiaryAttribute":"1.1.0","Payload":{"ProjectCategory":"CORE","SchemaVersion":"1.1.0"}},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"STAT","PrimaryAttribute":"Survey Statistics","SecondaryAttribute":None,"TertiaryAttribute":None,"Payload":{"MobileCompatible":True,"ID":"Survey Statistics"}},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"QC","PrimaryAttribute":"Survey Question Count","SecondaryAttribute":"2","TertiaryAttribute":None,"Payload":None},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"RS","PrimaryAttribute":"RS_cXYb9Itss4q723X","SecondaryAttribute":"Default Response Set","TertiaryAttribute":None,"Payload":None},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"SQ","PrimaryAttribute":"QID2","SecondaryAttribute":"Identify the word that doesn't belong:","TertiaryAttribute":None,"Payload":{"QuestionText":"Identify the word that doesn't belong:","DataExportTag":"News_19","QuestionType":"MC","Selector":"SAVR","SubSelector":"TX","Configuration":{"QuestionDescriptionOption":"UseText"},"QuestionDescription":"Identify the word that doesn't belong:","Choices":{"1":{"Display":"people"},"2":{"Display":"government"},"3":{"Display":"fbi"},"4":{"Display":"gun"},"5":{"Display":"fire"},"6":{"Display":"subject"}},"ChoiceOrder":[1,2,3,4,5,6],"Validation":{"Settings":{"ForceResponse":"OFF","ForceResponseType":"ON","Type":"None"}},"Language":[],"NextChoiceId":7,"NextAnswerId":1,"QuestionID":"QID2","GradingData":[],"Randomization":{"Advanced":None,"Type":"All","TotalRandSubset":""}}},{"SurveyID":"SV_af8BrDzrV6hF64d","Element":"SQ","PrimaryAttribute":"QID1","SecondaryAttribute":"Rate the Following Topic: people time told left started day home remember thought happened","TertiaryAttribute":None,"Payload":{"QuestionText":"Rate the Following Topic:<div><google-sheets-html-origin style=\"color: rgb(0, 0, 0); font-size: medium;\"><table class=\"UserTable\"><tbody><tr><td>people<\/td><\/tr><tr><td>time<\/td><\/tr><tr><td>told<\/td><\/tr><tr><td>left<\/td><\/tr><tr><td>started<\/td><\/tr><tr><td>day<\/td><\/tr><tr><td>home<\/td><\/tr><tr><td>remember<\/td><\/tr><tr><td>thought<\/td><\/tr><tr><td>happened<\/td><\/tr><\/tbody><\/table><\/google-sheets-html-origin><\/div>","DataExportTag":"Q1","QuestionType":"MC","Selector":"SAVR","SubSelector":"TX","Configuration":{"QuestionDescriptionOption":"UseText"},"QuestionDescription":"Rate the Following Topic: people time told left started day home remember thought happened","Choices":{"1":{"Display":"Useless"},"2":{"Display":"Neutral"},"3":{"Display":"Useful"}},"ChoiceOrder":["1","2","3"],"Validation":{"Settings":{"ForceResponse":"OFF","ForceResponseType":"ON","Type":"None"}},"Language":[],"NextChoiceId":4,"NextAnswerId":1,"QuestionID":"QID1"}}]}



In [ ]:
import json
print(json.dumps(survey, indent=2))

In [ ]:
print("hello")